In [4]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture('vtest.avi')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Define the output video writer
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (1280, 720))

# Read the first two frames
ret, frame1 = cap.read()
ret2, frame2 = cap.read()

if not ret or not ret2:
    print("Error: Unable to read frames from the video file.")
    cap.release()
    out.release()
    exit()

while cap.isOpened():
    # Check if both frames are valid
    if frame1 is None or frame2 is None:
        print("Error: One of the frames is None.")
        break

    # Compute the difference between the frames
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 900:
            continue

        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # Resize the frame and write it to the output
    image = cv2.resize(frame1, (1280, 720))
    out.write(image)

    # Display the frame
    cv2.imshow("feed", frame1)

    # Prepare for the next iteration
    frame1 = frame2
    ret, frame2 = cap.read()

    if not ret:  # Break if the next frame cannot be read
        print("End of video or error reading next frame.")
        break

    if cv2.waitKey(40) == 27:
        break

# Release resources
cv2.destroyAllWindows()
cap.release()
out.release()
